# AutoML in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Classification AutoML task.

**Motivations** - This notebook explains how to use Classification AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.automl import classification, regression
from azure.ai.ml.entities._job.automl.tabular import TabularFeaturizationSettings

# add environment variable to enable private preview feature
import os
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "true"


## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: C:\Users\vikasagrawal\azureml-examples\.azureml\config.json


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/sasik_rg/providers/Microsoft.MachineLearningServices/workspaces/sasik_ws_eastus2/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'c:\\Users\\vikasagrawal\\azureml-examples\\sdk\\jobs\\pipelines\\1h_automl_in_pipeline\\automl-classification-bankmarketing-in-pipeline', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001A6590AE440>, 'resource_id': None, 'location': 'eastus2', 'size': 'STANDARD_D2_V2', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 1800.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated'})


# 2. Basic pipeline job with Classification task

## 2.1 Build pipeline

In [4]:
# note that the used docker image doesn't suit for all size of gpu compute. Please use the following command to create gpu compute if experiment failed
# !az ml compute create -n gpu-cluster --type amlcompute --min-instances 0 --max-instances 4 --size Standard_NC12

In [5]:
# Define pipeline
@pipeline(
    description="AutoML Classification Pipeline",
    )
def automl_classification(
    classification_train_data,
    classification_validation_data
):
    # define the automl classification task with automl function
    classification_node = classification(
        training_data=classification_train_data,
        validation_data=classification_validation_data,
        target_column_name="y",
        primary_metric="accuracy",
        featurization=TabularFeaturizationSettings(mode="Auto"),
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits and training
    classification_node.set_limits(max_trials=1)
    classification_node.set_training(enable_stack_ensemble=False, enable_vote_ensemble=False)
    
    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path = Input(type="mlflow_model"),
            model_base_name= "bankmarketing_example_model",
            model_name_suffix= -1 # set negative to use epoch_secs
        ),
        outputs=dict(
             model_info_output_path= ""
        ),
        code="./register.py",
        command="python register.py " +
                "--model_input_path ${{inputs.model_input_path}} " +
                "--model_base_name ${{inputs.model_base_name}} " +
                "--model_name_suffix ${{inputs.model_name_suffix}} " +
                "--model_info_output_path ${{outputs.model_info_output_path}}",

        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    show_output = command_func(model_input_path = classification_node.outputs.best_model)
    


pipeline_classification = automl_classification(
    classification_train_data=Input(path="./training-mltable-folder/", type="mltable"),
    classification_validation_data=Input(path="./validation-mltable-folder/", type="mltable")
)

# set pipeline level compute
pipeline_classification.settings.default_compute="cpu-cluster"
        

Class ClassificationJob: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


# 2.2 Submit pipeline job

In [6]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_classification, experiment_name="pipeline_samples"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,blue_glass_nxtw2pfqc3,pipeline,Preparing,Link to Azure Machine Learning studio


In [7]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: blue_glass_nxtw2pfqc3
Web View: https://ml.azure.com/runs/blue_glass_nxtw2pfqc3?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sasik_rg/workspaces/sasik_ws_eastus2

Streaming logs/azureml/executionlogs.txt

[2022-08-10 11:22:29Z] Submitting 1 runs, first five are: 737068ba:c89cd2f7-34f3-477d-8726-b13367963303
[2022-08-10 11:29:23Z] Completing processing run id c89cd2f7-34f3-477d-8726-b13367963303.
[2022-08-10 11:29:23Z] Submitting 1 runs, first five are: 97145ca7:4ae72797-fad0-4477-8a8a-2172dfaf7f22
[2022-08-10 11:31:56Z] Completing processing run id 4ae72797-fad0-4477-8a8a-2172dfaf7f22.

Execution Summary
RunId: blue_glass_nxtw2pfqc3
Web View: https://ml.azure.com/runs/blue_glass_nxtw2pfqc3?wsid=/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/sasik_rg/workspaces/sasik_ws_eastus2



# Next Steps
You can see further examples of running a pipeline job [here](../)